<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/p018_padding_and_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.8.2


# Objectif
Manipuler les outils de tensorflow relatif au padding et au masking

In [3]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

### Padding Simple
Le moyen le plus simple est d'utiliser un layer padding `tf.keras.preprocessing.sequence.pad_sequences()` sur un tensor classique ou une liste


In [4]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


### Padding et Masking

Une fois que nos séquences sont padded, il faut être en mesure d'informer le modèle que certains tokens (ceux issus du padding) ne doivent pas être pris en compte lors du calcul de la loss. Autrement dit, il faut être capable de propager le masking de ces tokens. 3 Manières de le faire : 
 * Via un layer Masking `keras.layers.Masking`
   * Sous le capot, ce type de layer a un attribut `._keras_mask` qui représente le mask. `False` indique une valeur rembourée
 * Via un layer Embedding `keras.layers.Embedding` avec l'argument `mask_zero=True`
    * Sous le capot, ce type de layer a un attribut `._keras_mask` qui représente le mask. `False` indique une valeur rembourée
 * Manuellement lors du call des layers

In [11]:
embedding = tf.keras.layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)
print(padded_inputs)
print("")
print(masked_output.shape)
print(masked_output[-1])
print("")
print(masked_output._keras_mask)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]

(3, 6, 16)
tf.Tensor(
[[-0.00746113 -0.01262882  0.04603639  0.0171973  -0.01106896 -0.01516137
  -0.03219961  0.04006619  0.02056657 -0.00426289 -0.0479543  -0.02133725
  -0.04114251  0.02001885 -0.03566091 -0.03189234]
 [-0.03131612 -0.01921284 -0.04121875  0.03731486  0.0118412   0.02039745
  -0.00700837  0.04200404 -0.01960441  0.01622865 -0.04365989 -0.04678912
   0.0417749   0.04169219 -0.0342247   0.02585191]
 [ 0.02819878  0.03639678  0.01235443  0.02570499  0.04898168 -0.04376384
   0.04949386  0.02300647 -0.01527071 -0.01705949 -0.0031818  -0.03190134
   0.01958735  0.04385548 -0.0096609  -0.02756597]
 [-0.04424386 -0.02971691  0.04504538 -0.004831   -0.01226246 -0.01563602
  -0.0036243   0.04886606  0.02557868  0.03063506 -0.02767084 -0.03097909
   0.03716192 -0.0439463   0.02472824  0.01783756]
 [-0.02402784 -0.00328664 -0.02484034  0.00734135  0.00333361  0.01876425
   0.01

### Propagation du mask
Pour un des deux types de layers ci-dessus, le mask est propagé dans le réseau, tant que le layer suivant est capable de l'appliquer, j'imagine que ça signifie tant qu'il a une shape compatible. Ce processus fonctionne aussi bien sur l'API fonctionnelle que séquentielle. 

 * La couche embedding peut produire un mask. Elle possède une méthode `.compute_mask()` qui crée un mask
 * La couche LSTM peut alors utiliser le mask crée par l'embedding
 * self.embedding = `layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)`
 * `self.lstm = layers.LSTM(32)`
 * `mask = self.embedding.compute_mask(inputs)`
 * `output = self.lstm(x, mask=mask)`

In [ ]:
# Exemple d'un compute_mask method
def compute_mask(self, inputs, mask=None):
    if not self.mask_zero:
        return None
    return tf.not_equal(inputs, 0)

In [26]:
class TemporalSoftmax(tf.keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(
            inputs_exp * broadcast_float_mask, axis=-1, keepdims=True
        )
        return inputs_exp / inputs_sum


inputs = tf.keras.layers.Input(shape=(None,), dtype="int32")
x = tf.keras.layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = tf.keras.layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = tf.keras.Model(inputs, outputs)

In [27]:
indata = np.random.randint(0, 10, size=(32, 100))
outdata = np.random.random((32, 100, 1))
res = model(indata,outdata)

In [30]:
res[22]

<tf.Tensor: shape=(100, 1), dtype=float32, numpy=
array([[ 1.],
       [ 1.],
       [nan],
       [nan],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [nan],
       [nan],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [ 1.],
       [nan],
       [ 1.],
       [nan]